## Get meetup groups from Meetup, based on location

I got the idea and tips here: 
http://www.nesta.org.uk/blog/using-meetup-data-explore-uk-digital-tech-landscape

So, I looked into Meetups API http://www.meetup.com/meetup_api/ and wrote some code to do a similar job to the one above, except that I took the list of the tech meetups, the city, the amount of people they have and their organizer.

When I ran it in mid June 2016 this list had 2332 groups.

Basically, information like group size, meetups in a certain city, and so on, are now easily accessible.


#####instructions:

Get a list of cities in the country of interest, or just a list of cities:

In [2]:
import unicodecsv

def read_csv(filename): 
    with open(filename, 'rb') as f: 
        reader = unicodecsv.DictReader(f) 
        return list(reader) 

Got the list of cities from Wikipedia here: https://en.wikipedia.org/wiki/List_of_cities_in_the_United_Kingdom

In [3]:
cities_raw = read_csv('cities_raw.csv') 


Clean the cities up (my data had some numbers, brackets, and multiple names), 
help to make this better.

In [4]:
import pprint
cities = []
for city in cities_raw:
    end_char = len(city["city"])
    for i in "0123456789(": 
        f = city["city"].find(i)
        if f != -1 and end_char > f:
            end_char = f
    cities.append(str(city["city"][:end_char].replace("\n", "")))

pprint.pprint(cities)


['Aberdeen',
 'Armagh',
 'Bangor',
 'Bath',
 'Belfast',
 'Birmingham',
 'Bradford',
 'Brighton & Hove',
 'Bristol',
 'Cambridge',
 'Canterbury',
 'Cardiff',
 'Carlisle',
 'Chelmsford',
 'Chester',
 'Chichester',
 'Coventry',
 'Derby',
 'Derry / Londonderry',
 'Dundee',
 'Durham',
 'Edinburgh',
 'Ely',
 'Exeter',
 'Glasgow',
 'Gloucester',
 'Hereford',
 'Inverness',
 'Kingston upon Hull',
 'Lancaster',
 'Leeds',
 'Leicester',
 'Lichfield',
 'Lincoln',
 'Lisburn',
 'Liverpool',
 'City of London',
 'Manchester',
 'Newcastle upon Tyne',
 'Newport',
 'Newry',
 'Norwich',
 'Nottingham',
 'Oxford',
 'Perth',
 'Peterborough',
 'Plymouth',
 'Portsmouth',
 'Preston',
 'Ripon',
 'St Albans',
 'St Asaph',
 "St David's",
 'Salford',
 'Salisbury',
 'Sheffield',
 'Southampton',
 'Stirling',
 'Stoke-on-Trent',
 'Sunderland',
 'Swansea',
 'Truro',
 'Wakefield',
 'Wells',
 'City of Westminster',
 'Winchester',
 'Wolverhampton',
 'Worcester',
 'York']


Code to check if it is a valid location:

In [5]:
from dateutil.parser import parse
import requests

def valid_location(location):
    """Checks if the location gives lat and lng, asks till valid, returns it."""
    print location

    lat_lng_url = "http://maps.googleapis.com/maps/api/geocode/json?address=" + location +"%20UK"
    lat_lng_url_response = requests.get(lat_lng_url)
    location_data = lat_lng_url_response.json()
    if location_data["status"] == "OK":
        lat_lng = location_data["results"][0]["geometry"]["location"]
        lat, lng = lat_lng["lat"], lat_lng["lng"]
        return True, lat, lng
    else:
        return "No result for: " + location


Call the Meetup API

In [6]:
def get_groups(loc, offset):
    """Gets data with the help of the meetup API
    
    input: 
        loc: location data in this format [True/False, lat, long]
        offset: page only gives 200 meetups per call
    output: 
        groups_list: the list of all meetups (only the name, organizer, city, link, and number of members)
    """
    loc = valid_location(loc)
    groups_list = []
    if loc[0] == True:
        payload = {
            "key": "234968467c6b7e30115355f72575c",
            "lat": loc[1],
            "lon": loc[2],
            "offset": offset,
            "radius": "100", # 100 the most
            "category": "34", # tech category
            "order": "members",
        }

        api_res = requests.get("https://api.meetup.com/find/groups", params=payload)
        groups = api_res.json()

        for g in groups:
            meetup_name = g["name"]
            organizer = g["organizer"]["name"]
            city = g["city"]
            link = g["link"]
            members = g["members"]
            groups_list.append([meetup_name, link, city, organizer, members])
    else:
        groups_list.append(loc)

    return groups_list

In [7]:
endless_meetup_groups = []

def add_meetups(meet_list):
    for m in meet_list:
        if m not in endless_meetup_groups:
            endless_meetup_groups.append(m)

In [8]:
for city in cities:
    offset = 0
    print len(endless_meetup_groups)
    meetups = get_groups(city,offset)
    add_meetups(meetups)
    # page only gives 200 meetups per call, so if response has 200 meetups 
    # call again for same city with different offset
    max_groups_per_response = 200
    while len(meetups) ==  max_groups_per_response:
        offset +=1
        meetups = get_groups(city,offset)
        add_meetups(meetups)

0
Aberdeen
53
Armagh
Armagh
305
Bangor
Bangor
310
Bath
Bath
Bath
Bath
Bath
Bath
Bath
Bath
1816
Belfast
Belfast
1816
Birmingham
Birmingham
Birmingham
Birmingham
2136
Bradford
Bradford
2178
Brighton & Hove
Brighton & Hove
Brighton & Hove
Brighton & Hove
Brighton & Hove
Brighton & Hove
Brighton & Hove
Brighton & Hove
2291
Bristol
Bristol
2292
Cambridge
Cambridge
Cambridge
Cambridge
Cambridge
Cambridge
Cambridge
Cambridge
Cambridge
2309
Canterbury
Canterbury
Canterbury
Canterbury
Canterbury
Canterbury
Canterbury
Canterbury
2353
Cardiff
Cardiff
2357
Carlisle
2382
Chelmsford
Chelmsford
Chelmsford
Chelmsford
Chelmsford
Chelmsford
Chelmsford
Chelmsford
2382
Chester
Chester
2383
Chichester
Chichester
Chichester
Chichester
Chichester
Chichester
Chichester
Chichester
2383
Coventry
Coventry
Coventry
Coventry
Coventry
Coventry
Coventry
Coventry
Coventry
Coventry
2383
Derby
Derby
Derby
2384
Derry / Londonderry
2384
Dundee
2384
Durham
Durham
2384
Edinburgh
2384
Ely
Ely
Ely
Ely
Ely
Ely
Ely
Ely
2384
Ex

In [9]:
endless_meetup_groups

[[u'Scotland Data Science & Technology Meetup',
  u'http://www.meetup.com/Scotland-Data-Science-Technology-Meetup/',
  u'Edinburgh',
  u'Michael Young',
  1067],
 [u'Product Tank Edinburgh',
  u'http://www.meetup.com/ProductTankEdi/',
  u'Edinburgh',
  u'Simon Cast',
  652],
 [u'The Edinburgh Web Design & Development Meetup Group',
  u'http://www.meetup.com/The-Edinburgh-Web-Design-And-Development-Meetup-Group/',
  u'Edinburgh',
  u'David Smith',
  623],
 [u'iotEdinburgh (IOT//EDI)',
  u'http://www.meetup.com/iotEdinburgh/',
  u'Edinburgh',
  u'Simon Montford',
  585],
 [u'Edinburgh User Experience Meetup',
  u'http://www.meetup.com/Edinburgh-User-Experience-Meetup/',
  u'Edinburgh',
  u'Mike',
  482],
 [u'Edinburgh DevOps Meetup',
  u'http://www.meetup.com/Edinburgh-DevOps-Meetup/',
  u'Edinburgh',
  u'Paul Gillespie',
  459],
 [u'Lean Agile Edinburgh',
  u'http://www.meetup.com/Lean-Agile-Edinburgh/',
  u'Edinburgh',
  u'Stephen McAinsh',
  458],
 [u'Python Edinburgh',
  u'http://www

In [10]:
def getGroupSize(item):
    return int(item[4])

In [11]:
groups_by_size = sorted(endless_meetup_groups, key=getGroupSize, reverse=True)

In [26]:
groups_by_size

[[u'Silicon Roundabout',
  u'http://www.meetup.com/SiliconRoundabout/',
  u'London',
  u'Luis Carranza',
  10674],
 [u'London New Tech',
  u'http://www.meetup.com/London-New-Tech/',
  u'London',
  u'Marina Zd',
  8897],
 [u'Internet of Things London',
  u'http://www.meetup.com/iotlondon/',
  u'London',
  u'Alexandra Deschamps-Sonsino',
  8548],
 [u'HNLondon',
  u'http://www.meetup.com/HNLondon/',
  u'London',
  u'Dmitri Grabov',
  7599],
 [u'MiniBar',
  u'http://www.meetup.com/minibar/',
  u'London',
  u'Christian',
  7502],
 [u'Technopreneurs - Tech Startups and Entrepreneurs',
  u'http://www.meetup.com/technopreneurs/',
  u'London',
  u'Irfan Khalil',
  6964],
 [u'Data Science London',
  u'http://www.meetup.com/Data-Science-London/',
  u'London',
  u'Carlos',
  6741],
 [u'Virtual Java User Group',
  u'http://www.meetup.com/virtualJUG/',
  u'London',
  u'Simon Maple',
  6192],
 [u'Developers & Entrepreneurs',
  u'http://www.meetup.com/Developers-Entrepreneurs/',
  u'London',
  u'Rober

In [27]:
len(groups_by_size)

2396

In [29]:
import pandas as pd
meetups_by_size = pd.DataFrame(groups_by_size)
meetups_by_size.columns = ["meetup", "link", "city","organizer","members"]
meetups_by_size

,meetup,link,city,organizer,members
0,Silicon Roundabout,http://www.meetup.com/SiliconRoundabout/,London,Luis Carranza,10674
1,London New Tech,http://www.meetup.com/London-New-Tech/,London,Marina Zd,8897
2,Internet of Things London,http://www.meetup.com/iotlondon/,London,Alexandra Deschamps-Sonsino,8548
3,HNLondon,http://www.meetup.com/HNLondon/,London,Dmitri Grabov,7599
4,MiniBar,http://www.meetup.com/minibar/,London,Christian,7502
5,Technopreneurs - Tech Startups and Entrepreneurs,http://www.meetup.com/technopreneurs/,London,Irfan Khalil,6964
6,Data Science London,http://www.meetup.com/Data-Science-London/,London,Carlos,6741
7,Virtual Java User Group,http://www.meetup.com/virtualJUG/,London,Simon Maple,6192
8,Developers & Entrepreneurs,http://www.meetup.com/Developers-Entrepreneurs/,London,Robert Fenton,5667
9,LJC - London Java Community,http://www.meetup.com/Londonjavacommunity/,London,Barry Cranford,5550


Make a CSV with all the info for the meetup groups.

In [21]:
meetups_by_size.to_csv("meetups_uk.csv", sep='\t', encoding='utf-8')